In [1]:
import pandas as pd
import math
import re
import distance
import nltk
from datetime import datetime
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def tokenize(sent):
    """
    When passed in a sentence, tokenizes and normalizes the string,
    returning a list of lemmata.
    """
    lemmatizer = nltk.WordNetLemmatizer() 
    for token in nltk.wordpunct_tokenize(sent):
        token = token.lower()
        yield lemmatizer.lemmatize(token)

def normalized_jaccard(*args):
    try:
        return distance.jaccard(*[tokenize(arg) for arg in args])
    except UnicodeDecodeError:
        return 1.0
    
def wordsim(*args):
    return 1.0-normalized_jaccard(*args)



In [3]:
invoice = pd.read_excel("CCN 4189 FBL5N_Jul-Dec 2017.XLSX",\
                        dtype={'Reference':'str',\
                               'Document Number':'str',\
                               'Clearing Document':'str'})

invoice_1 = invoice[(invoice['Clearing date'] < '20171231') \
                    & (invoice['Posting Date'] > '20170701') \
                    & (invoice['Amount in doc. curr.'] > 0)]

invoice_1.head()

,Account,Account Name,Reference,Document Number,Text,User name,Company Code,Posting Date,Document Date,Baseline Payment Dte,Net due date,Document currency,Amount in doc. curr.,Local Currency,Amount in local currency,Local currency 2,Amount in loc.curr.2,Clearing date,Clearing Document
9,8364077,Z ENERGY 2015 LIMITED,127025,974947086,NaN,TCLT,4189,2017-07-04,2017-07-04,2017-07-04,2017-09-02,USD,82557.86,USD,82557.86,USD,82557.86,2017-09-05,14002573
10,8364077,Z ENERGY 2015 LIMITED,127099,974952978,NaN,TCLT,4189,2017-07-25,2017-07-25,2017-07-25,2017-09-23,USD,24960.00,USD,24960.00,USD,24960.00,2017-09-22,14000721
11,8364077,Z ENERGY 2015 LIMITED,127106,974954292,NaN,TCLT,4189,2017-07-31,2017-07-31,2017-07-31,2017-09-29,USD,53486.00,USD,53486.00,USD,53486.00,2017-09-29,14001766
12,8364077,Z ENERGY 2015 LIMITED,127149,974963972,NaN,TCLT,4189,2017-09-05,2017-09-05,2017-09-05,2017-11-04,USD,50240.00,USD,50240.00,USD,50240.00,2017-10-31,14000834
13,8364077,Z ENERGY 2015 LIMITED,127149,974968703,NaN,TCLT,4189,2017-09-05,2017-09-21,2017-09-05,2017-11-04,USD,320.00,USD,320.00,USD,320.00,2017-12-14,14002857


In [4]:
def write_to_excel(df,name):
    writer = pd.ExcelWriter(name,  datetime_format='MM/dd/yyyy')
    df.to_excel(writer,index=False)
    writer.close()

In [5]:
invoice_uncleared = invoice_1[invoice_1['Document Number']!=invoice_1['Clearing Document']]
print "There are {0} transactions in the invoice".format(len(invoice_1))
print "There are {0} transactions in the uncleared invoice".format(len(invoice_uncleared))

write_to_excel(invoice_uncleared,"CCN 4189 uncleared.xls")

There are 47102 transactions in the invoice
There are 46630 transactions in the uncleared invoice


In [6]:
set_document_uncleared = set(invoice_uncleared['Document Number'].values)

len(set_document_uncleared)

45920

In [7]:
bank = pd.read_excel("CCN 4189 Bank Statements_JPM_Oct-Dec 2017.xls",sheet_name="Details",\
                     dtype={'Remarks 1':'str','Remarks 2':'str','Remarks 3':'str','Remarks 4':'str',\
                            'Remarks 5':'str','Remarks 6':'str'})
bank.head()

,Bank ID,Bank Name,Currency,Account Number,Account Name,Transaction Date,Value Date,Description,Customer Reference,Bank Reference,...,Remarks 7,Remarks 8,Remarks 9,Remarks 10,Remarks 11,Remarks 12,Remarks 13,Remarks 14,Remarks 15,Transaction Time
0,02100002,JPMORGAN CHASE NEW YORK,USD,400944758,CHEVRON SINGAPORE PTE LTD - FAMM,2017-10-02,2017-10-02,BOOK TRANSFER CREDIT,SWF OF 17/10/02,1412600275HA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06:22 AM
1,02100002,JPMORGAN CHASE NEW YORK,USD,400944758,CHEVRON SINGAPORE PTE LTD - FAMM,2017-10-02,2017-10-02,CHIPS CREDIT,O/B DBTCO AMERIC,4000500275FC,...,CHIP SEQ =0017062,CHIP REF =201088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06:22 AM
2,02100002,JPMORGAN CHASE NEW YORK,USD,400944758,CHEVRON SINGAPORE PTE LTD - FAMM,2017-10-02,2017-10-02,CHIPS CREDIT,1IR710024595C01,2272800275FC,...,CHIP SEQ =0009109,CHIP REF =113192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:53 AM
3,02100002,JPMORGAN CHASE NEW YORK,USD,400944758,CHEVRON SINGAPORE PTE LTD - FAMM,2017-10-02,2017-10-02,CHIPS CREDIT,ST10501710020844,3492800275FC,...,CHIP SEQ =0010906,CHIP REF =176057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05:23 AM
4,02100002,JPMORGAN CHASE NEW YORK,USD,400944758,CHEVRON SINGAPORE PTE LTD - FAMM,NaT,NaT,Debit Summary,2 items,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
bank_1 = bank[(bank['Description'] != 'Debit Summary') & (bank['Description'] != 'Check Summary')]
print "There are {0} transactions in the bank statements".format(len(bank_1))
write_to_excel(bank_1,"CCN 4189 bank statements.xls")

There are 1079 transactions in the bank statements


In [9]:
def days_between(d1, d2):
    return abs(d2 - d1)/np.timedelta64(1, 'D')

In [27]:
n_customer=0
n_no_customer=0
n_match=0
lst_customer=[]
lst_name_match=[]
matches=[]

for index,row in bank_1[:200].iterrows():
    print "========================================================================="
    remarks = row['Remarks 1']+"|"+row['Remarks 2']+"|"+row['Remarks 3']+"|"+row['Remarks 4']+"|"+\
              row['Remarks 5']+"|"+row['Remarks 6']
    remarks = re.sub( '[\s\.]+', ' ',remarks).strip()
    remarks = remarks.replace('PETRON1/AS','PETRONAS')
    remarks = remarks.replace('PTE LT1','PTE LTD 1')
    remarks = remarks.replace('PTE LTDPAY','PTE LTD PAY')
    remarks = remarks.replace('PRIVATE LTD','PTE LTD')
    remarks = remarks.replace('LTD','LTD|')
    remarks = remarks.replace('PLC','PLC|')
    match_result = re.match(r".*B\/O CUSTOMER[\W\d\d]*([a-zA-Z\s\(\)\&]+).*",remarks)
    if match_result==None:
        print "no customer name: "+ remarks 
        n_no_customer+=1

    else:
        cust_name = match_result.group(1).replace("PTE","").replace("LTD","").replace("PLC","").strip()
        print"customer name found: "+cust_name
        n_customer+=1
        lst_customer.append(cust_name)
        amt = row['Credit Amount']
        date = row['Transaction Date']
        print "Transaction date: {1:'%Y-%m-%d'}, Credit amount: {0}, Customer Name: {2}".format(amt,date,cust_name)
        
        date_matched = invoice_1[(invoice_1['Net due date']>=date) & (invoice_1['Posting Date']<=date)]
        n_date_matched= len(date_matched)
        print "{0} invoices matches".format(n_date_matched)
        
        if(n_date_matched>0):
            match = date_matched.to_dict('records')
     
            for each in match:
                acc_inv = each['Account Name'].replace("PTE","").replace("LTD","").replace("PLC","").strip()
                amt_inv = each['Amount in doc. curr.']
                date_inv = each['Net due date']
                doc_inv = each['Document Number']
                name_score = wordsim(acc_inv,cust_name)
                amt_score = 1.0-abs(amt_inv/amt-1.0)
                date_score =  1.0-days_between(date,date_inv)/10.0
                final_score = name_score*0.18+amt_score*0.18+date_score*0.64
                

                if(final_score>0.9):
                    #print "name score: {0}  amount score: {1}  date score: {2} final score: {3}".\
                     #   format(name_score,amt_score,date_score,final_score)
                    print "invoice {3}: amount {0} from account {2} due on {1:'%Y-%m-%d'} final score: {4}".\
                        format(amt_inv,date_inv,acc_inv,doc_inv,final_score)
                    n_match+=1
                    matches.append({"Bank acc":cust_name,\
                                    "Bank date":date,"Bank amt":amt,\
                                    "Invoice acc":acc_inv,"Invoice amt":amt_inv,\
                                    "Invoice due":date_inv, "Name score":name_score,\
                                    "Amount score":amt_score,"Date score":date_score,\
                                    "Final score":final_score
                                    })
            
        else:
            print "no date range"

            #print date_matched[['Account Name','Net due date','Amount in doc. curr.']]
        

print "customer names found: {0}".format(n_customer)
print "customer names not found: {0}".format(n_no_customer)
print "final match count: {0}".format(n_match)

customer name found: HIN HIN TRADING
Transaction date: '2017-10-02', Credit amount: 1072538.46, Customer Name: HIN HIN TRADING
6151 invoices matches
invoice 78000256: amount 1072563.46 from account HIN HIN TRADING due on '2017-10-02' final score: 0.999995804346
customer name found: GLENCORE SINGAPORE
Transaction date: '2017-10-02', Credit amount: 927560.58, Customer Name: GLENCORE SINGAPORE
6151 invoices matches
invoice 78010644: amount 476495.5 from account GLENCORE SINGAPORE due on '2017-10-02' final score: 0.912467480668
invoice 78010646: amount 450635.08 from account GLENCORE SINGAPORE due on '2017-10-02' final score: 0.907449074647
customer name found: MERCURIA ENERGY TRADING
Transaction date: '2017-10-02', Credit amount: 629371.84, Customer Name: MERCURIA ENERGY TRADING
6151 invoices matches
invoice 78000282: amount 629371.84 from account MERCURIA ENERGY TRADING due on '2017-10-02' final score: 1.0
customer name found: SINOPEC FUEL OIL (SINGAPORE)
Transaction date: '2017-10-02', 

In [29]:
pd_matched = pd.DataFrame(matches)[['Bank acc','Bank date','Bank amt','Invoice acc','Invoice due','Invoice amt','Name score',\
                       'Date score','Amount score','Final score']]

write_to_excel(pd_matched,"matched_transactions.xls")

In [158]:
set(lst_customer)

{'AEGEAN MARINE PETROLEUM NETWORK IN',
 'AKR CORPORINDO PT',
 'ALLIANCE CONCRETE',
 'AMAZE CARCARE SERVICES PTE LTD',
 'AMERICAN EXPRESS INTERNATIONAL INC',
 'AMPOL SINGAPORE TRADING PTE LTD',
 'ANZ SG',
 'ARC ENERGY TRADING PTE LTD',
 'ASIAPAC TRADING PTE LTD',
 'AXIS FOCUS SDN BHD SUITE',
 'AXS PTE LTD',
 'BE',
 'BILSEA INTERNATIONAL PTE LTD',
 'BLACKHEM (M) SDN BHD UNIT NO',
 'BP INTERNATIONAL LTD',
 'BRIGHTOIL PETROLEUM (SPORE) PTE LTD',
 'BTS TANKERS PTE LIMITEDMAYBANK TOWER',
 'BUKIT BATOK DRIVING CENTRE LTD',
 'CABINDA GULF OIL COMPANY LIMITED',
 'CALTEX AUSTRALIA LIMITED TREASURY OPERATIONS LVL',
 'CH',
 'CHASSGSG',
 'CHEMOIL INTERNATIONALPTE LTD',
 'CHEV MARINE PRODUCTS LLC',
 'CHEVRON (CHINA) INVESTMENT CO LTUNIT',
 'CHEVRON (THAILAND ) LIMITED',
 'CHEVRON (TIANJIN) LUBRICANTS CONO',
 'CHEVRON CAMBODIA LIMITED',
 'CHEVRON HOLDINGS INC',
 'CHEVRON HOLDINGS INC TAIWAN BRANCH',
 'CHEVRON HONG KONG LTD',
 'CHEVRON INTERNATIONAL OIL CO',
 'CHEVRON INTERNATIONAL PTE LTD',
 'CHEVRON

In [21]:
for index,row in bank_1[:3].iterrows():
    print "========================================================================="
    remarks = row['Remarks 1']+row['Remarks 2']+row['Remarks 3']+row['Remarks 4']+row['Remarks 5']
    match_result = re.match(r".*=PRESENTER REF.(\d+).*",remarks)
    if match_result==None:
        print remarks 
        print row["Credit Amount"]
    else:
        ref = match_result.group(1)
        print"reference number found: "+ref
        print "invoice matched: " 
        for inv in invoice_1[invoice_1.Reference==ref].to_dict('records'):
            print inv

reference number found: 17526719
invoice matched: 
{u'Company Code': 4189, u'Clearing Document': '14002650', u'Account': 8217247, u'Reference': '17526719', u'Net due date': Timestamp('2017-10-02 00:00:00'), u'Text': u'Jaya Ocean-20170904-Asphalt BTB  2017-08Aug', u'Local currency 2': u'USD', u'Baseline Payment Dte': Timestamp('2017-10-02 00:00:00'), u'User name': u'XI_PC8008', u'Local Currency': u'USD', u'Document Number': '78000256', u'Amount in loc.curr.2': 1072563.46, u'Account Name': u'HIN HIN TRADING PTE LTD', u'Amount in doc. curr.': 1072563.46, u'Document Date': Timestamp('2017-09-04 00:00:00'), u'Document currency': u'USD', u'Clearing date': Timestamp('2017-10-02 00:00:00'), u'Amount in local currency': 1072563.46, u'Posting Date': Timestamp('2017-09-01 00:00:00')}
YOUR REF    =O/B DBTCO AMERICREC FROM    =DEUTSCHE BANK TRUST COMPANY AMERICAS 60 WALL STREET NEW YORK NYB/O CUSTOMER=/2016806055 GLENCORE SINGAPORE PTE LTD-BUNKER AND SEC 1 TEMASEK AVENUE 34-01 MILLENIA TOWERB/O BAN

In [87]:
m = re.match(r".*=PRESENTER REF.(\d+).*", "REMARK      =PRESENTER REF.17526719/HH007 /CHGS/USD25,00/ DEBIT REF 550-01-0223512")
m.group(1)

'17526719'

In [67]:
invoice_1[(invoice_1['Local Currency']=='USD') & (invoice_1['Amount in local currency']==1072538.46)]

,Account,Account Name,Reference,Document Number,Text,User name,Company Code,Posting Date,Document Date,Baseline Payment Dte,Net due date,Document currency,Amount in doc. curr.,Local Currency,Amount in local currency,Local currency 2,Amount in loc.curr.2,Clearing date,Clearing Document


In [12]:
set_invoice_ref = set(invoice_uncleared['Reference'].dropna().values)
set_bank_cust_ref = set(bank['Customer Reference'].dropna().values)
print "There are {0} unique ref. in the invoices".format(len(set_invoice_ref))
print "There are {0} unique customer ref. in the bank statements".format(len(set_bank_cust_ref))

print " {0:.2%} customers put invoice reference number in bank statements".format(len(set_bank_cust_ref)*1.0/len(set_invoice_ref))

There are 5096 unique ref. in the invoices
There are 680 unique customer ref. in the bank statements
 13.34% customers put invoice reference number in bank statements


In [13]:
def is_ref_exists(invoice_ref, cust_ref_set):
    return True in (invoice_ref in str(cust_ref) for cust_ref in cust_ref_set)

In [14]:
for ref in invoice['Document Number'].dropna().values:
    if is_ref_exists(str(ref),set_bank_cust_ref):
        print str(ref)

100060033
974972375
100060033
974964454
974968939
974972149
974977299
974981952
974984453
30000225
30000291


In [18]:
inv = invoice_uncleared[['Account Name','Amount in doc. curr.']]

In [16]:
bk = bank[['Credit Amount','Remarks 3']].dropna()

In [19]:
pd.merge(inv,bk,how="inner",left_on='Amount in doc. curr.',right_on='Credit Amount')

,Account Name,Amount in doc. curr.,Credit Amount,Remarks 3
0,Z ENERGY 2015 LIMITED,25280.00,25280.00,"B/O CUSTOMER=Z ENERGY LTD3 QUEENS WHARF, WELLI..."
1,VIETSEA COMPANY PTE LTD,600.00,600.00,PAY METHOD =INTERBANK GIRO
2,VIETSEA COMPANY PTE LTD,600.00,600.00,B/O CUSTOMER=BTS TANKERS PTE LIMITEDMAYBANK TO...
3,VIETSEA COMPANY PTE LTD,600.00,600.00,PAY METHOD =INTERBANK GIRO
4,VIETSEA COMPANY PTE LTD,600.00,600.00,PAY METHOD =INTERBANK GIRO
5,VIETSEA COMPANY PTE LTD,600.00,600.00,PAY METHOD =INTERBANK GIRO
6,STANDARD MARITIME PTE LTD,600.00,600.00,PAY METHOD =INTERBANK GIRO
7,STANDARD MARITIME PTE LTD,600.00,600.00,B/O CUSTOMER=BTS TANKERS PTE LIMITEDMAYBANK TO...
8,STANDARD MARITIME PTE LTD,600.00,600.00,PAY METHOD =INTERBANK GIRO
9,STANDARD MARITIME PTE LTD,600.00,600.00,PAY METHOD =INTERBANK GIRO
